In [211]:
import pandas as pd
import matplotlib.pyplot as plt

In [287]:
rt_movies_df = pd.read_csv("../data/rotten_tomatoes_movies.csv")
rt_series_df = pd.read_csv("../data/rotten_tomatoes_movies_with_money_2.csv")
rt_series_df = rt_series_df[rt_series_df.wd_film_qid.notna() & (rt_series_df.series_qid.notna() | rt_series_df.franchise_qid.notna())]


rt_series_df['n_films_in_series'] = rt_series_df.groupby(['series_qid']).transform('size')
rt_series_df['n_films_in_franchise'] = rt_series_df.groupby(['franchise_qid']).transform('size')
rt_series_df = rt_series_df.query("n_films_in_series > 1 or n_films_in_franchise > 1")

print(rt_series_df.shape)
metadata_df = pd.read_csv("../data/clean_tmdb.csv")
rt_series_df = rt_series_df.merge(rt_movies_df, on="rotten_tomatoes_link", how="inner")

print(rt_series_df.shape)
rt_series_df = rt_series_df.merge(metadata_df[['budget','revenue','imdb_id']], left_on="imdb_id", right_on="imdb_id", how="left")
rt_series_df = rt_series_df.rename(columns={"budget":"imdb_budget", "revenue":"imdb_box_office_value"})
print(rt_series_df.shape)

rt_series_df = rt_series_df.merge(metadata_df[['budget','revenue','id']], left_on="tmdb_id", right_on="id", how="left")
rt_series_df = rt_series_df.rename(columns={"budget":"tmdb_budget", "revenue":"tmdb_box_office_value"})
print(rt_series_df.shape)

(1250, 23)
(1250, 44)
(1250, 46)
(1250, 49)


In [264]:
print("missing budget:", rt_series_df.budget_value.isna().sum())
print("filled by imdb:", (rt_series_df.budget_value.isna() & rt_series_df.imdb_budget.notna()).sum())
print("filled by tmdb:", (rt_series_df.budget_value.isna() & rt_series_df.imdb_budget.isna() & rt_series_df.imdb_budget.notna()).sum())
print("still missing:", (rt_series_df.budget_value.isna() & rt_series_df.imdb_budget.isna() & rt_series_df.tmdb_budget.isna()).sum())

missing budget: 530
filled by imdb: 335
filled by tmdb: 0
still missing: 195


In [288]:
print("missing box_office_value:", rt_series_df.box_office_value.isna().sum())
print("filled by imdb:", (rt_series_df.box_office_value.isna() & rt_series_df.imdb_box_office_value.notna()).sum())
print("filled by tmdb:", (rt_series_df.box_office_value.isna() & rt_series_df.imdb_box_office_value.isna() & rt_series_df.tmdb_box_office_value.notna()).sum())
print("still missing:", (rt_series_df.box_office_value.isna() & rt_series_df.imdb_box_office_value.isna() & rt_series_df.tmdb_box_office_value.isna()).sum())

missing box_office_value: 516
filled by imdb: 300
filled by tmdb: 0
still missing: 216


In [291]:
box_office_missing = (rt_series_df.box_office_value.isna() & rt_series_df.imdb_box_office_value.isna() & rt_series_df.tmdb_box_office_value.isna())
budget_missing = (rt_series_df.budget_value.isna() & rt_series_df.imdb_budget.isna() & rt_series_df.tmdb_budget.isna())
print("one is missing: ", ((box_office_missing) | (budget_missing)).sum())

print("was missing:", (rt_series_df.box_office_value.isna() | rt_series_df.budget_value.isna()).sum())

one is missing:  222
was missing: 604


In [290]:
rt_series_df[((box_office_missing) | (budget_missing))][['wd_film_label', 'original_release_date', 'imdb_id','tmdb_id']].sort_values(by="original_release_date", ascending=False)[:50]

,wd_film_label,original_release_date,imdb_id,tmdb_id
186,"Berlin, I Love You",2019-02-08,tt2275743,401686.0
749,Once Upon a Deadpool,2018-12-12,tt31050020,567604.0
635,Leprechaun Returns,2018-12-11,tt8155182,518158.0
84,A Christmas Prince: The Royal Wedding,2018-11-30,tt8709036,525041.0
834,Puppet Master: The Littlest Reich,2018-08-17,tt5685006,396321.0
1160,To All the Boys I've Loved Before,2018-08-17,tt3846674,466282.0
1109,The Monkey King 3,2018-02-16,tt6466464,437543.0
82,A Christmas Prince,2017-11-17,tt7608418,483104.0
738,Ocean Waves,2016-12-28,tt0108432,21057.0
780,Phantasm: Ravager,2016-10-07,tt3627704,262848.0


In [244]:
print(len(metadata_df.id))
print(metadata_df.id.nunique())

print(len(metadata_df[metadata_df.imdb_id.notna()].imdb_id))
print(metadata_df[metadata_df.imdb_id.notna()].imdb_id.nunique())

16013
16013
15527
15526


In [255]:
rt_series_df.query("imdb_id == 'tt0455857'")

,rotten_tomatoes_link,wd_film_qid,wd_film_label,series_qid,series_label,franchise_qid,franchise_label,bom_film_id,imdb_id,tmdb_id,...,tomatometer_count,audience_status,audience_rating,audience_count,tomatometer_top_critics_count,tomatometer_fresh_critics_count,tomatometer_rotten_critics_count,imdb_budget,tmdb_budget,id
1362,m/when_a_stranger_calls,Q908384,When a Stranger Calls,Q99642278,When a Stranger Calls,NaN,NaN,whenastrangercalls06,tt0455857,10053.0,...,93.0,Spilled,44.0,477438.0,18,8,85,15000000.0,15000000.0,10053.0


In [ ]:
rt_series_df = rt_series_df.merge(metadata_df[["imdb_id"]])

,rotten_tomatoes_link,wd_film_qid,wd_film_label,series_qid,series_label,franchise_qid,franchise_label,bom_film_id,imdb_id,tmdb_id,...,production_company,tomatometer_status,tomatometer_rating,tomatometer_count,audience_status,audience_rating,audience_count,tomatometer_top_critics_count,tomatometer_fresh_critics_count,tomatometer_rotten_critics_count
0,m/0814255,Q244333,Percy Jackson & the Olympians: The Lightning T...,Q7167417,Percy Jackson & the Olympians,NaN,NaN,percyjackson,tt0814255,32657.0,...,20th Century Fox,Rotten,49.0,149.0,Spilled,53.0,254421.0,43,73,76
1,m/1000617-aliens,Q104814,Aliens,Q1990792,Alien,Q1990792,Alien,aliens,tt0090605,679.0,...,20th Century Fox,Certified-Fresh,97.0,76.0,Upright,94.0,429674.0,17,74,2
2,m/10008587-beverly_hills_chihuahua,Q390341,Beverly Hills Chihuahua,Q4899550,Beverly Hills Chihuahua,NaN,NaN,beverlyhillschihuahua,tt1014775,14405.0,...,Walt Disney Pictures,Rotten,40.0,99.0,Spilled,52.0,186570.0,29,40,59
3,m/10009132-rec,Q754474,Rec,Q22908176,REC,NaN,NaN,NaN,tt1038988,8329.0,...,Odeon Sky Filmworks,Fresh,89.0,37.0,Upright,82.0,50437.0,5,34,4
4,m/10009926-alvin_and_the_chipmunks_2,Q378311,Alvin and the Chipmunks: The Squeakquel,Q4738263,Alvin and the Chipmunks in film,Q917295,Alvin and the Chipmunks,alvinandthechipmunksii,tt1231580,23398.0,...,20th Century Fox,Rotten,20.0,84.0,Spilled,49.0,368619.0,30,17,67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1401,m/young_guns,Q1353728,Young Guns,Q28970837,Young Guns,NaN,NaN,youngguns,tt0096487,11967.0,...,Artisan Entertainment,Rotten,41.0,37.0,Upright,76.0,62440.0,9,15,22
1402,m/young_guns_2,Q883213,Young Guns II,Q28970837,Young Guns,NaN,NaN,youngguns2,tt0100994,9086.0,...,20th Century Fox,Rotten,35.0,17.0,Upright,66.0,52462.0,3,6,11
1403,m/yu_gi_oh_the_movie,Q962917,Yu-Gi-Oh! The Movie: Pyramid of Light,NaN,NaN,Q192015,Yu-Gi-Oh!,yugioh,tt0403703,11052.0,...,Warner Bros. Pictures,Rotten,5.0,66.0,Spilled,49.0,10998.0,22,3,63
1404,m/zodiac,Q218172,Zodiac,Q26705935,BBC's 100 Greatest Films of the 21st Century,NaN,NaN,zodiac07,tt0443706,1949.0,...,Paramount Pictures,Certified-Fresh,89.0,253.0,Upright,77.0,494309.0,60,225,28


In [208]:
metadata_df[metadata_df.imdb_id.duplicated(keep=False)][['title', 'imdb_id']]

,title,imdb_id
9734,Gangs of Wasseypur - Part 2,NaN
10987,Box,NaN
11279,I am,NaN
12366,2020 (G)I-DLE Online Concert 'I-Land : Who Am I',NaN
12419,RED,NaN
...,...,...
16000,Writer's block,NaN
16005,The It's Pinky Movie,NaN
16007,One More Round,NaN
16010,Skeye Hallelujah Cover,NaN


In [ ]:
rt_series_df = rt_series_df.merge(metadata_df[['id', 'budget']], left_on='tmdb_id', right_on='id', how='left')
rt_series_df = rt_series_df.merge(metadata_df[['tmdb_', '']], left_on='tmdb_id', right_on='id', how='left', suffixes=('', '_tmdb'))
rt_series_df = rt_series_df.rename(columns={'budget': 'tmdb_budget'})
rt_series_df
# rt_series_df = rt_series_df.drop(columns=['title'])

,rotten_tomatoes_link,wd_film_qid,wd_film_label,series_qid,series_label,franchise_qid,franchise_label,bom_film_id,imdb_id,tmdb_id,...,tomatometer_rating,tomatometer_count,audience_status,audience_rating,audience_count,tomatometer_top_critics_count,tomatometer_fresh_critics_count,tomatometer_rotten_critics_count,id,tmdb_budget
0,m/0814255,Q244333,Percy Jackson & the Olympians: The Lightning T...,Q7167417,Percy Jackson & the Olympians,NaN,NaN,percyjackson,tt0814255,32657.0,...,49.0,149.0,Spilled,53.0,254421.0,43,73,76,32657.0,95000000.0
1,m/1000617-aliens,Q104814,Aliens,Q1990792,Alien,Q1990792,Alien,aliens,tt0090605,679.0,...,97.0,76.0,Upright,94.0,429674.0,17,74,2,679.0,18500000.0
2,m/10008587-beverly_hills_chihuahua,Q390341,Beverly Hills Chihuahua,Q4899550,Beverly Hills Chihuahua,NaN,NaN,beverlyhillschihuahua,tt1014775,14405.0,...,40.0,99.0,Spilled,52.0,186570.0,29,40,59,14405.0,20000000.0
3,m/10009132-rec,Q754474,Rec,Q22908176,REC,NaN,NaN,NaN,tt1038988,8329.0,...,89.0,37.0,Upright,82.0,50437.0,5,34,4,8329.0,1500000.0
4,m/10009926-alvin_and_the_chipmunks_2,Q378311,Alvin and the Chipmunks: The Squeakquel,Q4738263,Alvin and the Chipmunks in film,Q917295,Alvin and the Chipmunks,alvinandthechipmunksii,tt1231580,23398.0,...,20.0,84.0,Spilled,49.0,368619.0,30,17,67,23398.0,75000000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1401,m/young_guns,Q1353728,Young Guns,Q28970837,Young Guns,NaN,NaN,youngguns,tt0096487,11967.0,...,41.0,37.0,Upright,76.0,62440.0,9,15,22,11967.0,13000000.0
1402,m/young_guns_2,Q883213,Young Guns II,Q28970837,Young Guns,NaN,NaN,youngguns2,tt0100994,9086.0,...,35.0,17.0,Upright,66.0,52462.0,3,6,11,9086.0,20000000.0
1403,m/yu_gi_oh_the_movie,Q962917,Yu-Gi-Oh! The Movie: Pyramid of Light,NaN,NaN,Q192015,Yu-Gi-Oh!,yugioh,tt0403703,11052.0,...,5.0,66.0,Spilled,49.0,10998.0,22,3,63,11052.0,20000000.0
1404,m/zodiac,Q218172,Zodiac,Q26705935,BBC's 100 Greatest Films of the 21st Century,NaN,NaN,zodiac07,tt0443706,1949.0,...,89.0,253.0,Upright,77.0,494309.0,60,225,28,1949.0,65000000.0


In [186]:
print("commons: ", rt_series_df[rt_series_df.budget_value.isna() & rt_series_df.tmdb_budget.notna()].shape[0])
print("filled by tmdb: rt_series_df[rt_series_df.budget_value.isna()")

commons:  374
filled by tmdb: rt_series_df[rt_series_df.budget_value.isna()


In [109]:
rt_series_df[rt_series_df.budget_value.isna() & rt_series_df.budget_from_tmdb.notna()]

,rotten_tomatoes_link,wd_film_qid,wd_film_label,series_qid,series_label,franchise_qid,franchise_label,budget_value,budget_currency_qid,budget_currency,...,tomatometer_status,tomatometer_rating,tomatometer_count,audience_status,audience_rating,audience_count,tomatometer_top_critics_count,tomatometer_fresh_critics_count,tomatometer_rotten_critics_count,budget_from_tmdb
4,m/10009926-alvin_and_the_chipmunks_2,Q378311,Alvin and the Chipmunks: The Squeakquel,Q4738263,Alvin and the Chipmunks in film,Q917295,Alvin and the Chipmunks,NaN,NaN,NaN,...,Rotten,20.0,84.0,Spilled,49.0,368619.0,30,17,67,75000000.0
5,m/10010667-hangover,Q219315,The Hangover,Q1587838,The Hangover,NaN,NaN,NaN,NaN,NaN,...,Certified-Fresh,78.0,239.0,Upright,84.0,943179.0,51,187,52,35000000.0
8,m/1004047-children_of_the_corn,Q467953,Children of the Corn,Q4159357,Children of the Corn,NaN,NaN,NaN,NaN,NaN,...,Rotten,35.0,26.0,Spilled,40.0,84960.0,0,9,17,12000000.0
10,m/1005512-death_in_venice,Q742308,Death in Venice,Q108741934,Q108741934,NaN,NaN,NaN,NaN,NaN,...,Fresh,71.0,24.0,Upright,81.0,6409.0,4,17,7,3500000.0
20,m/1011617-king_kong,Q1142862,King Kong,NaN,NaN,Q3876200,King Kong,NaN,NaN,NaN,...,Rotten,53.0,40.0,Spilled,31.0,57614.0,7,21,19,30000000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1385,m/wrong_turn_2,Q1166161,Wrong Turn 2: Dead End,Q3166407,Wrong Turn,NaN,NaN,NaN,NaN,NaN,...,Fresh,70.0,10.0,Spilled,44.0,28390.0,0,7,3,15000000.0
1397,m/yella,Q1372318,Yella,Q17615030,Ghost-Trilogy,NaN,NaN,NaN,NaN,NaN,...,Certified-Fresh,80.0,54.0,Spilled,53.0,974.0,20,43,11,120000000.0
1401,m/young_guns,Q1353728,Young Guns,Q28970837,Young Guns,NaN,NaN,NaN,NaN,NaN,...,Rotten,41.0,37.0,Upright,76.0,62440.0,9,15,22,150000000.0
1402,m/young_guns_2,Q883213,Young Guns II,Q28970837,Young Guns,NaN,NaN,NaN,NaN,NaN,...,Rotten,35.0,17.0,Upright,66.0,52462.0,3,6,11,180000000.0


In [119]:
rt_series_df.loc[1316]

rotten_tomatoes_link                              m/transformers_the_last_knight_2017
wd_film_qid                                                                 Q24251026
wd_film_label                                           Transformers: The Last Knight
series_qid                                                                   Q2073466
series_label                                                             Transformers
franchise_qid                                                                     NaN
franchise_label                                                                   NaN
budget_value                                                              217000000.0
budget_currency_qid                                                             Q4917
budget_currency                                                                   NaN
budget_asof                                                                       NaN
budget_source                                         

In [110]:
rt_series_df[rt_series_df.budget_value.isna() & rt_series_df.budget_from_tmdb.isna()].sort_values(by='original_release_date', ascending=False)

,rotten_tomatoes_link,wd_film_qid,wd_film_label,series_qid,series_label,franchise_qid,franchise_label,budget_value,budget_currency_qid,budget_currency,...,tomatometer_status,tomatometer_rating,tomatometer_count,audience_status,audience_rating,audience_count,tomatometer_top_critics_count,tomatometer_fresh_critics_count,tomatometer_rotten_critics_count,budget_from_tmdb
1327,m/trolls_world_tour,Q29825830,Trolls World Tour,Q97185752,Trolls,Q97185752,Trolls,NaN,NaN,NaN,...,Fresh,70.0,152.0,Spilled,59.0,1087.0,30,107,45,NaN
145,m/annabelle_comes_home,Q55973412,Annabelle Comes Home,Q104771487,Annabelle,NaN,NaN,NaN,NaN,NaN,...,Fresh,64.0,203.0,NaN,70.0,9416.0,29,130,73,NaN
707,m/leprechaun_returns,Q60048020,Leprechaun Returns,Q1763934,Leprechaun,NaN,NaN,NaN,NaN,NaN,...,Rotten,44.0,9.0,Spilled,48.0,127.0,0,4,5,NaN
1207,m/the_grinch,Q15632949,The Grinch,Q65120698,list of Illumination films,NaN,NaN,NaN,NaN,NaN,...,Fresh,60.0,190.0,Spilled,50.0,12908.0,29,114,76,NaN
1298,m/to_all_the_boys_ive_loved_before,Q35783117,To All the Boys I've Loved Before,Q85809868,To All the Boys,NaN,NaN,NaN,NaN,NaN,...,Certified-Fresh,96.0,68.0,Upright,86.0,4688.0,13,65,3,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1122,m/superman_red_son,Q78914620,Superman: Red Son,Q2405799,DC Universe Animated Original Movies,NaN,NaN,NaN,NaN,NaN,...,Fresh,88.0,16.0,Upright,64.0,344.0,1,14,2,NaN
1143,m/teen_titans_the_judas_contract,Q28465798,Teen Titans: The Judas Contract,Q27703173,DC Animated Movie Universe,NaN,NaN,NaN,NaN,NaN,...,Fresh,83.0,6.0,Upright,73.0,802.0,0,5,1,NaN
1163,m/the-cheetah-girls-2,Q587876,The Cheetah Girls 2,Q7722302,The Cheetah Girls,NaN,NaN,NaN,NaN,NaN,...,Rotten,57.0,7.0,Upright,61.0,66166.0,3,4,3,NaN
1299,m/to_all_the_boys_ps_i_still_love_you,Q62761144,To All the Boys: P.S. I Still Love You,Q85809868,To All the Boys,NaN,NaN,NaN,NaN,NaN,...,Certified-Fresh,75.0,71.0,Spilled,40.0,500.0,15,53,18,NaN


In [89]:
rt_series_df.query("wd_film_label.str.contains('Grinch')")

,rotten_tomatoes_link,wd_film_qid,wd_film_label,series_qid,series_label,franchise_qid,franchise_label,budget_value,budget_currency_qid,budget_currency,...,tomatometer_status,tomatometer_rating,tomatometer_count,audience_status,audience_rating,audience_count,tomatometer_top_critics_count,tomatometer_fresh_critics_count,tomatometer_rotten_critics_count,budget_from_tmdb
1207,m/the_grinch,Q15632949,The Grinch,Q65120698,list of Illumination films,NaN,NaN,NaN,NaN,NaN,...,Fresh,60.0,190.0,Spilled,50.0,12908.0,29,114,76,NaN


In [188]:
metadata_df.columns

Index(['id', 'title', 'vote_average', 'vote_count', 'status', 'release_date',
       'revenue', 'runtime', 'adult', 'backdrop_path', 'budget', 'homepage',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'tagline', 'genres',
       'production_companies', 'production_countries', 'spoken_languages',
       'keywords'],
      dtype='object')

In [118]:
rt_series_df.merge(metadata_df[['title', 'budget']], left_on='wd_film_label', right_on='title', how='left', indicator=True)['budget'].loc[1316]

np.float64(75000000.0)